#  JW Paper Replicate

In [64]:
import pandas as pd
import numpy as np
import os
os.getcwd()


'/Users/JiangmengZhang/Documents/ml-finance'

## LM list + combine inflection (stemming)

In [158]:
def read_lm(txt): 
    with  open(txt) as f:
        lines = f.readlines()
    new_lines = [x for x in lines if not x=='\n']
    new_lines = [x[:-1] for x in new_lines]
    return(new_lines)

In [159]:
pos = read_lm("LM_Positive.txt")
neg = read_lm("LM_Negative.txt")

In [200]:
 from nltk.stem.porter import PorterStemmer
 porter = PorterStemmer() # could be Snowball stemmer or Lancaster stemmer
 def tokenizer_porter(lm):
    return ([porter.stem(word) for word in lm])
 pos_stem = tokenizer_porter(pos)
 neg_stem = tokenizer_porter(neg) 

In [201]:
from nltk.stem import WordNetLemmatizer
lemmas = WordNetLemmatizer()
def tokenizer_lemmas(lm):
    return([lemmas.lemmatize(word.lower(), pos = 'v') for word in lm])
pos_lem = tokenizer_lemmas(pos)
neg_lem = tokenizer_lemmas(neg)


In [211]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
def tokenize_doc(doc):
#doc = "A paragraph from the Ancient Greek paragraphos able excellent, to write beside or written beside) is a self-contained unit of a discourse in writing dealing with a particular point or idea. A paragraph consists of one or more sentences."
    doc = doc.lower()
    wordlist = doc.split()
    ## remove no's neighbour
    noidx = [i for i,x in enumerate(wordlist) if x in ["no", "not", "never"]]
    noidx2 = []
    for one in noidx:
        noidx2.append(range(one-3, one+4))
    noidx2 = list(set(noidx2))
    wordlist2 = [wordlist[i] for i in range(len(wordlist)) if not i in noidx2] 

    ## word count
    wordcount = Counter(wordlist2)
    posneg = pos + neg
    # posneg_lem = pos_lem + neg_lem
    posneg_stem = pos_stem + neg_stem
    posneglower = [item.lower() for item in posneg]
    wordcount2 = {x:wordcount[x] for x in wordcount.keys() if x in posneglower}
    wordcount2 = {x:wordcount2.get(x,0) for x in posneglower} #filling 0
    wordcount3 = dict()
    for i,x in enumerate(posneg_stem):
        wordcount3[x] = wordcount3.get(x, 0) + wordcount2[posneg2[i]] # combine
    return(len(wordlist), wordcount3)

## 10K
1. remove stop words and proper nouns
2. do not count positive or negative words that are accompanied by a "negator" (not, no, never) with a distance of 3 words
3. tokenize

## Stock Price
CRSP value-weight index return?

Organize all data into one dataframe

In [ ]:
## "release":10K release date;  "company"; "ret": abnormal return; "len": # of word; "wordlist": tokenlized wordlist; 
df = pd.DataFrame()
for doc in docs:
    res = tokenize_doc(doc)
    df = df.append(res[1].values()/res[0])
    
df['ret'] = 

## Regression of return on word to get weight and score

In [ ]:
from sklearn import linear_model
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
lm = linear_model.LinearRegression()
lm.fit(X, y)
lm.coef_

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

## Regression of return on score

In [ ]:
pos_idx = range(len(set(pos_stem)))
neg_idx = range(len(set(pos_stem)), len(set(posneg_stem)))
score_pos = X.iloc[:, pos_idx]*lm.coef_[pos_idx]
score_neg = X.iloc[:, neg_idx]*lm.coef_(neg_idx)
Xscore = pd.DateFrame()
Kscore['pos'] = score_pos
Xscore['neg'] = score_neg

Xscore2 = sm.add_constant(X)
est_score = sm.OLS(y, Xscore2)
est_score2 = est_score.fit()
print(est_score2.summary())

Future works
1. collocation
2. training vs testing

Question:
1. Note my original (very old) suggestion was to use map (which is much slower):
df1['e'] = df1['a'].map(lambda x: np.random.random())

2. why red above?
3. return 2 variables 
4. df.values?